# Truth Testing in Python

In Python, everything is `True` except for the following:

* `None` value
* `False` value
* Any zero values: 0 (int), 0.0 (float)
* Empty collections: list, tuple, set, dict, ...
* Objects with `__nonezero__()` method which returns `False`
* Objects with `__len__` method which returns 0

For more information, see the [reference](https://docs.python.org/2/library/stdtypes.html?highlight=truth%20value%20testing#truth-value-testing)

# Boolean Tips

|             Do            |             Don't             |                       Note                       |
|---------------------------|-------------------------------|--------------------------------------------------|
| if a == b                 | if (a == b)                   | Python do not require parentheses                |
| myvar is None             | myvar == None                 | Faster                                           |
| myvar not in my_collection | not (myvar in my_collection)   | Shorter, more pythonic                           |
| if 1 <= myvar <= 10       | if 1 <= myvar and myvar <= 10 | Shorter, more beautiful                          |
| if my_collection           | if len(my_collection) == 0     | Sometimes you prefer the second form for clarity |
| if all(a, b, c)           | if a and b and c              | Shorter                                          |
| if any(a, b, c)           | if a or b or c                | Shorter, clearer                                 |


# Dealing with Complex Boolean Expressions

When a boolean expression gets long and complicated, it helps to factor it out to a separate function. The name of the function alone should help reducing the complexity and keep the reader focused. For example, consider the following code segment:

    if float(dev.version) == float(version) and dev.language == language \
                                            and dev.locale == locale and dev.model == model:
                                            
If we factor this expression out so the code becomes:

    if device_match(dev, version, language, locale, model):
    
    ...
    def device_match(dev, version, language, locale, model):
        """
        Match a device against a set of attribute, return True if they match,
        False otherwise.
        """
        expected = (float(version), language, locale, model)
        actual = (float(dev.version), dev.language, dev.locale, dev.model)
        return expected == actual

---

Another example, consider the following code:

    if expected_row_count is None and expected_column_count is None and \
            expected_data_table is None and excluded_strings is None and included_strings is None:
        raise TypeError("At least one argument for validation should not be None")

which can be rewritten as:

    parameters = [
        expected_row_count,
        expected_column_count,
        expected_data_table,
        excluded_strings,
        included_strings]
        
    if all(param is None for param in parameters):

... or

    if parameters == [None, None, None, None, None]:
    
... or

    if parameters == [None] * 5:
    
---

Consider this code segnment:

    #find a session that starts after the test start time or
    # cases where desktop session starts first and test starts later
    if (session_start_time > test_start_time) or \
            (session_start_time < test_start_time and session_end_time > test_start_time):

The logic here is complex enough to ask for a refactor:


    if should_record_log(test_start_time, session_start_time, session_end_time):
    
Not only the rewrite is cleaner, it also communicates the purpose of this if block via the name of the function.

# String Predicates

String class has a number of useful predicates, for examples

    mystr.startswith('Sheet')
    mystr.endswith('ing')
    'Sheet' in mystr
    mystr.isalnum()  # 0-9, a-z, A-Z
    mystr.isalpha()  # a-z, A-Z
    mystr.isdigit()  # 0-9
    mystr.isspace()  # \t, \n, \r
    mystr.islower()  # a-z
    mystr.isupper()  # A-Z


# Recipe to Eliminate If #1: Loop through a Collection

Consider the following function which prints a list of elements from a collection. For some reason, we allow the collection to be None and is optional:

In [1]:
def print_collection(my_collection=None):
    if my_collection is None:
        my_collection = []
    print('<ul>')
    for element in my_collection:
        print('  <li>{}</li>'.format(element))
    print('</ul>')
    
print_collection(['John', 'Paul', 'George', 'Ringo'])

<ul>
  <li>John</li>
  <li>Paul</li>
  <li>George</li>
  <li>Ringo</li>
</ul>


When the collection is None (or omitted):

In [2]:
print_collection()

<ul>
</ul>


The function above works fine, but I don't like the `if` statement and would like to eliminate it using the `or` trick:

In [3]:
def print_collection(my_collection=None):
    print('<ul>')
    for element in my_collection or []:
        print('  <li>{}</li>'.format(element))
    print('</ul>')
    
print_collection(['John', 'Paul', 'George', 'Ringo'])

<ul>
  <li>John</li>
  <li>Paul</li>
  <li>George</li>
  <li>Ringo</li>
</ul>


The expression `my_collection or []` works like this: If my_collection is `None` then the `or` expression will continue to evaluate the next term, the empty list and return that empty list. Looping through an empty list will bypass it. Here is the case when `my_collection` is `None`:

In [4]:
print_collection()

<ul>
</ul>


# Recipe to Eliminate If #2: Toggle a Flag

If we have a boolean flag and want to toggle it, the most straight-forward way is:

In [5]:
flag = True  # Initial
# ... sometime later
if flag:
    flag = False
else:
    flag = True

Believe it or not, I saw this many times in my career. A better, shorter way is:

In [6]:
flag = not flag

# Recipe to Eliminate If #3: Cycle of Values

If we have a value and want to cycle within [0..N], the straight-forward solution is:

In [7]:
# Initial
N = 4
myvar = 0

# ... sometime later
if myvar == 4:
    myvar = 0
else:
    myvar += 1

Again, we can eliminate the if block by using the modulus operator:

In [8]:
myvar = (myvar + 1) % N

# Recipe to Eliminate If #4: Cycle of Values

A more complex version of the clycing of values has to do with non-numeric (or non-consecutive numeric) values:

In [9]:
colors = ['red', 'green', 'blue']
for i in range(6):
    print(colors[i % len(colors)])

red
green
blue
red
green
blue


While the previous code segment already eliminated the `if` statement using the modulus trick we learned earlier, it is far from being readable. A better way is to use `itertools.cycle`:

In [2]:
import itertools
colors = ['red', 'green', 'blue']
colors_cycle = itertools.cycle(colors)
for _ in range(6):
    print(next(colors_cycle))

red
green
blue
red
green
blue


Note that the `cycle` function, as with some other functions in the `itertools` module, are infinite generators. That means they will keep on generating values as long as the caller keep requesting. That means we need a way to put a stop or the loop will run indefinitely. By ziping `range(6)` with the `cycle` generator, we achieve that goal.